## Look at every data set currently in Agora and get a list of Ensembl IDs that are either missing from the original gene_table_merged file or the updated gene_table_merged file.

In [1]:
import pandas as pd
import synapseclient
import sys

sys.path.insert(0, '../../')

import agoradatatools.etl.utils as utils
import agoradatatools.etl.extract as extract

#### The two gene lists we are using for comparison. 

In [2]:
syn = synapseclient.Synapse()
syn.login(silent=True)

# The gene file currently used in Agora. Uncomment to use this file
#gene_table_old = extract.get_entity_as_df(syn_id='syn25953363',
#                                          format='feather',
#                                          syn=syn)

# The original RData file, generated by R code, converted to a feather file
# This original file has more information filled in than the file currently used in 
# Agora, because the database was queried in 2018 for this file, rather than in 
# 2020/2021 on the same set of genes like the file currently being used. 
gene_table_old = pd.read_feather('../input/gene_table_merged_orig_RData.feather')

gene_table_new = extract.get_entity_as_df(syn_id='syn38973224',
                                          format='feather',
                                          syn=syn)

print(gene_table_old.shape)
print(gene_table_new.shape)

(60725, 8)
(68326, 9)


#### Get file list
We will use the configuration in test_config.yaml to fetch all the files. This file is almost identical to config.yaml, but ```overall_scores``` and ```distribution_data``` are enabled in the test_config file. 

File used at the time of this script's development is test_config.yaml from [commit 101f1843daa898cb3c8f92df6e6ba5ff7a5988e1](https://github.com/Sage-Bionetworks/agora-data-tools/blob/101f1843daa898cb3c8f92df6e6ba5ff7a5988e1/test_config.yaml).

In [3]:
config = utils._get_config(config_path="../../test_config.yaml")
datasets = config[1]['datasets']

files = {}

for dataset in datasets:
    dataset_name = list(dataset.keys())[0]
    
    for entity in dataset[dataset_name]['files']:
        entity_id = entity['id']
        entity_format = entity['format']
        entity_name = entity['name']
        
        # Ignore json files, which are post-processed and not what we're interested in. 
        # Also ignore the gene_info file as that's what we loaded in above. 
        if entity_format != 'json' and entity_name != 'gene_info':
            files[entity_name] = [entity_id, entity_format]

files

{'neuropath_regression_results': ['syn22017882', 'csv'],
 'agora_proteomics': ['syn18689335', 'csv'],
 'agora_proteomics_tmt': ['syn35221005', 'csv'],
 'target_exp_validation_harmonized': ['syn24184512', 'csv'],
 'srm_data': ['syn25454540', 'csv'],
 'metabolomics': ['syn26064497', 'feather'],
 'igap': ['syn12514826', 'csv'],
 'eqtl': ['syn12514912', 'csv'],
 'proteomics': ['syn18689335', 'csv'],
 'rna_expression_change': ['syn27211942', 'tsv'],
 'target_list': ['syn12540368', 'csv'],
 'median_expression': ['syn27211878', 'csv'],
 'druggability': ['syn13363443', 'csv'],
 'team_info': ['syn12615624', 'csv'],
 'team_member_info': ['syn12615633', 'csv'],
 'overall_scores': ['syn25575156', 'table'],
 'networks': ['syn11685347', 'csv'],
 'diff_exp_data': ['syn27211942', 'tsv'],
 'proteomics_tmt': ['syn35221005', 'csv']}

#### Remove duplicate synapse IDs

In [4]:
files_unique = {}
ids = []
for F in files.keys():
    if files[F][0] not in ids:
        ids.append(files[F][0])
        files_unique[F] = files[F]

print(len(files))
print(len(files_unique))

files = files_unique

19
16


#### We should now have a list of all raw data files ingested. Get each one and compare it to the gene lists. 

In [5]:
missing_list_old = {}
missing_list_new = {}

# The various column names used to store Ensembl IDs in the files
col_names = ['ENSG', 'ensembl_gene_id', 'GeneID']

for file in files.keys():
    df = extract.get_entity_as_df(syn_id=files[file][0],
                                  format=files[file][1],
                                  syn=syn)
    
    ensembl_ids = None
    
    for C in col_names:
        if C in df.columns:
            ensembl_ids = df[C]
    
    # networks file is a special case
    if file == "networks":
        ensembl_ids = pd.melt(df[['geneA_ensembl_gene_id','geneB_ensembl_gene_id']])['value']
        
    if ensembl_ids is not None:
        diffs = set(ensembl_ids) - set(gene_table_old['ensembl_gene_id'])
        if len(diffs) > 0:
            missing_list_old[file] = list(diffs)
        
        diffs2 = set(ensembl_ids) - set(gene_table_new['ensembl_gene_id'])
        if len(diffs2) > 0:
            missing_list_new[file] = list(diffs2)
        
        
print(missing_list_old.keys())
print(missing_list_new.keys())
    

dict_keys(['neuropath_regression_results', 'agora_proteomics', 'agora_proteomics_tmt', 'target_exp_validation_harmonized', 'metabolomics', 'igap', 'eqtl', 'rna_expression_change', 'target_list', 'median_expression', 'druggability', 'overall_scores'])
dict_keys(['neuropath_regression_results', 'agora_proteomics', 'agora_proteomics_tmt', 'target_exp_validation_harmonized', 'metabolomics', 'eqtl', 'rna_expression_change', 'median_expression', 'druggability', 'overall_scores', 'networks'])


In [6]:
print([[X, len(missing_list_old[X])] for X in missing_list_old.keys()])
print('\n')
print([[X, len(missing_list_new[X])] for X in missing_list_new.keys()])

all_old = set(sum(missing_list_old.values(), []))
all_new = set(sum(missing_list_new.values(), []))

missing_list_old['combined'] = list(all_old)
missing_list_new['combined'] = list(all_new)

print('\n')
print(len(all_old))
print(len(all_new))

[['neuropath_regression_results', 359], ['agora_proteomics', 9], ['agora_proteomics_tmt', 5], ['target_exp_validation_harmonized', 1], ['metabolomics', 31], ['igap', 7], ['eqtl', 1], ['rna_expression_change', 1016], ['target_list', 1], ['median_expression', 1016], ['druggability', 677], ['overall_scores', 949]]


[['neuropath_regression_results', 46], ['agora_proteomics', 4], ['agora_proteomics_tmt', 3], ['target_exp_validation_harmonized', 1], ['metabolomics', 909], ['eqtl', 190], ['rna_expression_change', 105], ['median_expression', 105], ['druggability', 66], ['overall_scores', 100], ['networks', 147]]


1753
1132


#### Find lost information
Agora currently only uses genes that are in 'the list' and ignores genes in the data files that are not in the list. So the actual "lost" information from switching lists would be genes that are in the 'missing list' AND in the old gene list. Based on the output below, it looks like we were actually losing more genes by NOT updating the list. 

In [7]:
in_old_not_new = {}

for K in missing_list_new.keys():
    # If it's in missing_new and in the old gene list
    missing = [X for X in missing_list_new[K] if X in set(gene_table_old['ensembl_gene_id'])]
    if len(missing) > 0 and missing[0] != 'n/A':
        in_old_not_new[K] = missing

print('Lost by switching to new list:')
print([[X, len(in_old_not_new[X])] for X in in_old_not_new.keys()])

in_new_not_old = {}

for K in missing_list_old.keys():
    # If it's in missing_old and in the new gene list
    missing = [X for X in missing_list_old[K] if X in set(gene_table_new['ensembl_gene_id'])]
    if len(missing) > 0 and missing[0] != 'n/A':
        in_new_not_old[K] = missing

print('\nLost by keeping old list:')
print([[X, len(in_new_not_old[X])] for X in in_new_not_old.keys()])

Lost by switching to new list:
[['neuropath_regression_results', 38], ['agora_proteomics', 4], ['agora_proteomics_tmt', 3], ['metabolomics', 878], ['eqtl', 189], ['rna_expression_change', 94], ['median_expression', 94], ['druggability', 64], ['overall_scores', 89], ['networks', 147], ['combined', 1085]]

Lost by keeping old list:
[['neuropath_regression_results', 351], ['agora_proteomics', 9], ['agora_proteomics_tmt', 5], ['igap', 7], ['rna_expression_change', 1005], ['target_list', 1], ['median_expression', 1005], ['druggability', 675], ['overall_scores', 938], ['combined', 1706]]


Available gene info (**old** table) for genes that are missing from the **new** list

In [8]:
df = gene_table_old.loc[gene_table_old['ensembl_gene_id'].isin(in_old_not_new['combined'])]
print(df[['ensembl_gene_id', 'symbol']])

       ensembl_gene_id      symbol
46     ENSG00000273547        None
219    ENSG00000216045        None
285    ENSG00000280851        None
293    ENSG00000280816        None
295    ENSG00000236269        None
...                ...         ...
59577  ENSG00000184258        CDR1
59632  ENSG00000221870        None
59856  ENSG00000261773  AC244090.3
59881  ENSG00000277203        F8A1
59901  ENSG00000274791        F8A2

[1085 rows x 2 columns]


#### How many of these actually have a gene name, and therefore at least some useful information?

In [9]:
df = gene_table_old.loc[gene_table_old['ensembl_gene_id'].isin(in_old_not_new['combined']) \
                        & gene_table_old['symbol'].notnull()]

print(len(df))
print(df[['ensembl_gene_id', 'symbol']])

print('\nThis affects:')
print([K for K in in_old_not_new.keys() if any(item in in_old_not_new[K] for item in df['ensembl_gene_id'])])


257
       ensembl_gene_id      symbol
360    ENSG00000277726  AL109811.4
362    ENSG00000271895  AL109811.3
651    ENSG00000225986  UBXN10-AS1
710    ENSG00000271840  AL031281.2
1171   ENSG00000116883  AL591845.1
...                ...         ...
59037  ENSG00000261409  AL035425.3
59577  ENSG00000184258        CDR1
59856  ENSG00000261773  AC244090.3
59881  ENSG00000277203        F8A1
59901  ENSG00000274791        F8A2

[257 rows x 2 columns]

This affects:
['neuropath_regression_results', 'agora_proteomics', 'agora_proteomics_tmt', 'metabolomics', 'eqtl', 'rna_expression_change', 'median_expression', 'druggability', 'overall_scores', 'networks', 'combined']


Available gene info (**new** table) for genes that are missing from the **old** list

In [10]:
df = gene_table_new.loc[gene_table_new['ensembl_gene_id'].isin(in_new_not_old['combined'])]
print(df[['ensembl_gene_id', 'symbol']])

       ensembl_gene_id  symbol
4070   ENSG00000112459  OR14J1
4071   ENSG00000112461   OR5V1
7466   ENSG00000137345     MOG
12379  ENSG00000168468   ATF6B
17041  ENSG00000196231   OR2J2
...                ...     ...
66447  ENSG00000288110    None
66854  ENSG00000288520    None
66970  ENSG00000288656    None
67005  ENSG00000288701  PRRC2B
67006  ENSG00000288702  UGT1A3

[1706 rows x 2 columns]


#### How many of these actually have a gene name, and therefore at least some useful information?

In [11]:
df = gene_table_new.loc[gene_table_new['ensembl_gene_id'].isin(in_new_not_old['combined']) \
                        & gene_table_new['symbol'].notnull()]

print(len(df))
print(df[['ensembl_gene_id', 'symbol']])

print('\nThis affects:')
print([K for K in in_new_not_old.keys() if any(item in in_new_not_old[K] for item in df['ensembl_gene_id'])])

870
       ensembl_gene_id        symbol
4070   ENSG00000112459        OR14J1
4071   ENSG00000112461         OR5V1
7466   ENSG00000137345           MOG
12379  ENSG00000168468         ATF6B
17041  ENSG00000196231         OR2J2
...                ...           ...
66349  ENSG00000288011     LOC283045
66386  ENSG00000288049  LOC105377224
66399  ENSG00000288062  LOC101929748
67005  ENSG00000288701        PRRC2B
67006  ENSG00000288702        UGT1A3

[870 rows x 2 columns]

This affects:
['neuropath_regression_results', 'agora_proteomics', 'agora_proteomics_tmt', 'igap', 'rna_expression_change', 'target_list', 'median_expression', 'druggability', 'overall_scores', 'combined']


#### Write the missing Ensembl IDs to files

In [12]:
pd.DataFrame(missing_list_old['combined']).to_csv("../output/missing_list_old.csv")
pd.DataFrame(missing_list_new['combined']).to_csv("../output/missing_list_new.csv")
pd.DataFrame(in_old_not_new['combined']).to_csv("../output/in_old_not_new.csv")
pd.DataFrame(in_new_not_old['combined']).to_csv("../output/in_new_not_old.csv")

#### Summary
While we lose 1085 gene IDs total by switching to the new list, only a subset of those genes actually have associated information and would provide any useful information in Agora. 

If you run this using the data file currently used in Agora (syn25953363), only 18 informational genes are "lost". 

If you run this using the original RData file (syn15666826, converted to feather for this script), 257 informational genes are "lost". However most of these genes were *already* lost when the new feather file (syn25953363) was created due to querying the mygene database with a several-years-old gene list. 


On the other hand, by **not** switching to the new list, we are losing 870 genes with associated information. 